In [1]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [2]:
SEASONS = list(range(2023, 2025))

In [3]:
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [4]:
async def get_html(url, selector, sleep = 5, retries = 3):
    html = None
    for i in range(1, retries + 1):
        time.sleep(sleep * i)

        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [5]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    href = [l["href"] for l in links]
    standings_pages = [f"https://basketball-reference.com{l}" for l in href]

    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
    
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [6]:
for season in SEASONS:
    await scrape_season(season)

2022-23 NBA Schedule | Basketball-Reference.com
2023-24 NBA Schedule | Basketball-Reference.com


In [7]:
standings_files = os.listdir(STANDINGS_DIR)

In [8]:
async def scrape_game(standings_file):
    with open(standings_file, "r") as f:
        html = f.read()
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get("href") for l in links]
    box_scores = [l for l in hrefs if l and "boxscores" in l and ".html" in l]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores]
    
    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
    
        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [9]:
standings_files = [s for s in standings_files if ".html" in s]


In [ ]:
for f in standings_files:
    filepath = os.path.join(STANDINGS_DIR, f)

    await scrape_game(filepath)

Jazz vs Heat, March 2, 2024 | Basketball-Reference.com
Trail Blazers vs Grizzlies, March 2, 2024 | Basketball-Reference.com
Nuggets vs Lakers, March 2, 2024 | Basketball-Reference.com
Rockets vs Suns, March 2, 2024 | Basketball-Reference.com
76ers vs Mavericks, March 3, 2024 | Basketball-Reference.com
Warriors vs Celtics, March 3, 2024 | Basketball-Reference.com
Clippers vs Timberwolves, March 3, 2024 | Basketball-Reference.com
Pistons vs Magic, March 3, 2024 | Basketball-Reference.com
Hornets vs Raptors, March 3, 2024 | Basketball-Reference.com
Knicks vs Cavaliers, March 3, 2024 | Basketball-Reference.com
Pacers vs Spurs, March 3, 2024 | Basketball-Reference.com
Thunder vs Suns, March 3, 2024 | Basketball-Reference.com
Grizzlies vs Nets, March 4, 2024 | Basketball-Reference.com
Clippers vs Bucks, March 4, 2024 | Basketball-Reference.com
Trail Blazers vs Timberwolves, March 4, 2024 | Basketball-Reference.com
Wizards vs Jazz, March 4, 2024 | Basketball-Reference.com
Bulls vs Kings, Marc